<a href="https://colab.research.google.com/github/jbgour/Music-Generation-Project/blob/main/music_generation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


créé un dossier music-generation-colab dans ton gg drive et mets y le dossier data

In [44]:
pwd

'/content/drive/My Drive/music-generation-project/data'

In [31]:
import os
os.chdir("/content/drive/My Drive/music-generation-project")

In [2]:
#import os
#os.chdir("drive/My Drive/music-generation-colab")

FileNotFoundError: ignored

In [14]:
import pandas as pd
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


def prepare_sequences(notes, n_vocab, sequence_length = 100):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = sequence_length

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output, epochs=2, batch_size=128):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=epochs,
              batch_size=batch_size, callbacks=callbacks_list)




In [45]:
cd ..

/content/drive/My Drive/music-generation-project


In [46]:
ls

data/  music-generation-colab.ipynb


In [91]:
notes = []
log = []

annees = [2009]
notes_source = []
for x in annees:
    notesAnnees = pd.read_pickle(r'data/results/notes_' + str(x))
    notes_source += notesAnnees
notes_source = notes_source[0:1000]


# test avec entrée = alternance de 2 notes


In [92]:
notes = notes_source
n_vocab = len(set(notes))
print(notes[0:10])

['F#3', 'E4.C#4', 'A4.A5.C#5', 'A3', 'C#3', 'E3', 'F#1', 'G#3', 'A1', 'F#1']


In [93]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab, sequence_length=10)


print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output,epochs=100, batch_size=128)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/100
8/8 [==============================] - 4s 50ms/step - loss: 4.8707
Epoch 2/100
8/8 [==============================] - 0s 42ms/step - loss: 4.4427
Epoch 3/100
8/8 [==============================] - 0s 41ms/step - loss: 4.3875
Epoch 4/100
8/8 [==============================] - 0s 46ms/step - loss: 4.3850
Epoch 5/100
8/8 [==============================] - 0s 41ms/step - loss: 4.3765
Epoch 6/100
8/8 [==============================] - 0s 45ms/step - loss: 4.3691
Epoch 7/100
8/8 [==============================] - 0s 41ms/step - loss: 4.3317
Epoch 8/100
8/8 [==============================] - 0s 46ms/step - loss: 4.3584
Epoch 9/100
8/8 [==============================] - 0s 39ms/step - loss: 4.2722
Epoch 10/100
8/8 [==============================] - 0s 43ms/step - loss: 4.2286
Epoch 11/100
8/8 [==============================] - 0s 39ms/step - loss: 4.3827
Epoch 12/100
8/8 [==============================] - 0s 38ms/step - los

In [94]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 10 notes
for note_index in range(20):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index/ float(n_vocab))
    pattern = pattern[1:len(pattern)]
    
print(prediction_output)

generating notes
['F#5', 'G#5', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4', 'F#5', 'B-4', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4', 'F#5', 'B-4', 'C4', 'G5.E5']


In [95]:
pattern = network_input[start]


In [ ]:
!pip install midiutil

In [ ]:
!pip install music21

In [96]:
def pitch_to_midi_number(sequence, pattern):
    modi_pattern=[int_to_note[round(pattern[i][0]*float(n_vocab))] for i in range(len(pattern))]
    print(modi_pattern)
    full_seq=modi_pattern+sequence
    print(full_seq)
    midi_notes = []
    for p in full_seq:
        if ('.' in p):
            notes_in_chord = p.split('.')
            notes = []
            for current_note in notes_in_chord:
                notes.append(pitch.Pitch(current_note).midi)
            midi_notes.append(notes)
        else:
            midi_notes.append(pitch.Pitch(p).midi)
    return midi_notes

In [97]:
from midiutil import MIDIFile
from music21 import *

degrees  = pitch_to_midi_number(prediction_output, pattern)  # MIDI note number
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 100   # In BPM
volume   = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                      # automatically)
MyMIDI.addTempo(track, time, tempo)

for i, pitch in enumerate(degrees):
    if type(pitch) is list:
        for p in pitch:
            MyMIDI.addNote(track, channel, p, time + i, duration, volume)
    else:
        MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

with open("major-scale.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

['E-3.F#2', 'E-2.F#1', 'F#5', 'B-4', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4']
['E-3.F#2', 'E-2.F#1', 'F#5', 'B-4', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4', 'F#5', 'G#5', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4', 'F#5', 'B-4', 'C4', 'G5.E5', 'G#2', 'F#6', 'E-3.F#2', 'C#4', 'F#5', 'B-4', 'C4', 'G5.E5']
